In [ ]:
import hydromt
import xarray as xr
import numpy as np
import pandas as pd
from os.path import join, basename
import glob
import geopandas as gpd
import pygeos
import matplotlib.pyplot as plt

In [ ]:
bbox =  32.0, -21.5, 35.5, -17.0
cat = hydromt.DataCatalog(deltares_data=True)


In [ ]:
## read AVISO MDT

da_mdt = cat.get_rasterdataset('mdt_cnes_cls18', bbox=bbox)

In [ ]:
## CODEC
root = r'p:\11205283-hydromt-floodmodelling\00_data'
chunks={'stations': 100, 'time':-1}
rm = {'station_x_coordinate':'lon', 'station_y_coordinate':'lat'}


In [ ]:
# name = 'CODEC'
# fns = glob.glob(join(root, name, 'reanalysis_waterlevel_10min_*_v1_beira.nc'))
# ds_gtsm0 = xr.open_dataset(fns[0], chunks=chunks).rename(rm).vector.clip_bbox(bbox)
# gdf_gtsm = ds_gtsm0.vector.to_gdf().set_crs(4326)

In [ ]:
## IDAI / ELOISE
events = {
    # 'idai': dict(
    #     name = 'GTSM_20190218_20190401_IDAI',
    #     dates = ('2019-01-01', '2019-05-01')
    # ),
    'eloise': dict(
        name = 'GTSM_20201218_20210201_ELOISE',
        dates = ('2021-01-01', '2021-03-01')
    
    )
}
var = 'waterlevel'
encoding = {var: {'dtype': 'float32'}}
runs = ['era5', 'spw', 'tides', 'era5_tides', 'spw_tides', 'era5_spw_tides'][1:]

In [ ]:
# select stations within bbox
prefix = next(iter(events))
fn0 = join(root, events[prefix]['name'], f'{prefix}_{runs[0]}_his.nc')
da_gtsm0 = xr.open_dataset(fn0, chunks=chunks).rename(rm)[var]
da_gtsm0['stations'] = da_gtsm0.stations
da_gtsm0 = da_gtsm0.vector.clip_bbox(bbox)
da_gtsm0.raster.set_crs(4326)
gdf_gtsm = da_gtsm0.vector.to_gdf()
gdf_gtsm.to_file(join(root, 'gtsm_beira.geojson'), driver='GeoJSON')
da_gtsm0.station_name.values

In [ ]:
import os

mdt0 = da_mdt.raster.sample(gdf_gtsm).rename({'index': 'stations'}).reset_coords(drop=True)
mdt0.name = 'mdt'
run = ''

for prefix, event in events.items():
    print(event)
    name = event['name']
    tstart, tstop = event['dates']
    # reindex to get timeseries from JAN-01
    time = xr.IndexVariable('time', pd.date_range(tstart, tstop, freq='10MIN'))

    # read htot
    fn_htot = join(root, name, f'{prefix}_era5_spw_tides_his.nc')
    da_htot = xr.open_dataset(fn_htot, chunks=chunks)[var].sel(stations=gdf_gtsm.index)
    da_htot = da_htot.resample(time='10MIN').nearest().reindex(time=time, fill_value=0)
    # read htide
    fn_htide = join(root, name, f'{prefix}_tides_his.nc')
    da_htide = xr.open_dataset(fn_htide, chunks=chunks)[var].sel(stations=gdf_gtsm.index)
    da_htide = da_htide.resample(time='10MIN').nearest().reindex(time=time, fill_value=0)
    # read waves
    da_shww0 = cat.get_rasterdataset('era5_hourly', bbox=bbox, variables=['shww'], time_tuple=(tstart, tstop))
    da_shww = da_shww0.raster.sample(gdf_gtsm).rename({'index': 'stations'}).reset_coords(drop=True)
    da_shww = da_shww.reindex(time=time, method='nearest').load()
    da_shww.name = 'shww'
    
    # combine sl components
    # write htot referenced to EGM
    da_htot1 = mdt0 + da_htot + da_shww*0.2
    da_htot1.name = var
    da_surge1 = da_htot - da_htide
    da_surge1.name = 'surge'
    da_htide.name = 'tide'
    fn_out = fn_htot.replace('.nc', '_beira_egm.nc')
    ds_out = xr.merge([da_htot1, da_htide, da_surge1, da_shww, mdt0]).transpose('time','stations')
    encoding1 = {v: {'zlib': True} for v in ds_out.data_vars}
    print(basename(fn_out))
    ds_out.to_netcdf(fn_out, encoding=encoding1)

    # write htide referenced to EGM
    da_htide1 = mdt0 + da_htide
    da_htide1.name = var
    da_htide1 = da_htide1.transpose('time','stations')
    fn_out = fn_htide.replace('.nc', '_beira_egm.nc')
    print(basename(fn_out))
    da_htide1.to_netcdf(fn_out, encoding=encoding)
    
    # sensitivity analysis on surge + waves
    for mult in [0.8, 1.2]:  # sensitivity
        da_htot2 = mdt0 + (da_htot + da_shww*0.2)*mult
        da_htot2.name = var
        da_htot2 = da_htot2.transpose('time','stations')
        fn_out1 = fn_htot.replace('.nc', f'_beira_egm_x{int(mult*100)}.nc')
        print(basename(fn_out1))
        da_htot2.to_netcdf(fn_out1, encoding=encoding)      
        
    

In [ ]:
import hydromt

events = {'idai': ('20190314', '20190316'), 'eloise':('20210122', '20210124')}
event ='eloise'
fn = f'../../1_data/2_forcing/GTSM/{event}_era5_spw_tides_his_beira_egm.nc'
ds = xr.open_dataset(fn).sel(time=slice(*events[event])).dropna('stations')
ds.vector.set_spatial_dims(x_dim='station_x_coordinate', y_dim='station_y_coordinate', index_dim='stations')
itime = ds['waterlevel'].argmax('time')
# ds.isel(time=itime).vector.to_gdf().drop(columns=['station_name', 'time']).to_file(fn.replace('.nc', '.geojson'), driver='GeoJSON')
ds['shww'] = ds['shww']*0.2
ds['waterlevel'] = ds['waterlevel'] - ds['mdt']
ds0 = ds.isel(time=itime).sel(stations=[17492, 17493, 18275, 18276])
(ds0['shww'] / (ds0['tide'] + ds0['surge'] + ds0['shww'])).mean()
ds0

In [ ]:
ds.sel(stations=[17492, 17493, 18275, 18276]).max('time')